In [14]:
import requests
from bs4 import BeautifulSoup
import re
import string
import json

def get_var(text, var):
    """
    :type text: str
    :type var: str
    :rtype: str
    """
    pattern = 'var\s+' + var.rstrip() + '\s+?=\s+?{'
    open_token_found = False
    block = '{'
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue
        if open_token_found:
            if re.match('};', line):
                block += '}'
                break
            else:
                segments = line.split(':', 1)
                key = segments[0]
                if key[0] != '"':
                    key = '"' + key
                if key[-1] != '"':
                    key = key + '"'
                block += key + ':' + segments[1]
        elif re.match(pattern, line):
                open_token_found = True

    if block[-2] == ',':
        block = block[:-2] + '}'

    # edge case: O'Hearn throws JSONDecodeError
    if 'O"' in block:
        block = block.replace('O"',"O'")

    return json.loads(block)

# function that takes in a pitchers baseball savant pitching profile and parses out various stats/data.
def parse_bs_pitcher_script(url):

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # find the javascript section based on unique identifier: var serverVals =
    script = [script.text for script in soup.find_all('script') if 'var serverVals =' in script.text][0].replace("'",'"').replace("null","0")
    
    data_dict = get_var(script, 'serverVals')

    return data_dict
if __name__ == '__main__':

    _ = parse_bs_pitcher_script('https://baseballsavant.mlb.com/savant-player/gerrit-cole-543037?stats=statcast-r-pitching-mlb')